In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [3]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('acgan')):
    os.mkdir('saved_models_mnist_{}'.format('acgan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('acgan'), mode='w')
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
import numpy as np
import torch.utils.data as Data
class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = mnist.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)
        # Rescale -1 to 1
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                # Image labels. 0-9
                img_labels = y_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # The labels of the digits that the generator tries to create an
                # image representation of
                sampled_labels = np.random.randint(0, 10, (batch_size, 1))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, sampled_labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, [valid, img_labels])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss[0]))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise,sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=30, batch_size=64, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 8, 32)          0         
__________

epoch:0 step:49 [D loss: 2.411057, acc.: 82.03%, op_acc: 37.50%] [G loss: 4.049882]
epoch:0 step:50 [D loss: 2.468952, acc.: 85.16%, op_acc: 25.78%] [G loss: 3.780288]
epoch:0 step:51 [D loss: 2.280949, acc.: 90.62%, op_acc: 36.72%] [G loss: 3.884015]
epoch:0 step:52 [D loss: 2.457361, acc.: 83.59%, op_acc: 32.81%] [G loss: 3.819124]
epoch:0 step:53 [D loss: 2.428313, acc.: 82.81%, op_acc: 30.47%] [G loss: 3.971150]
epoch:0 step:54 [D loss: 2.249285, acc.: 81.25%, op_acc: 37.50%] [G loss: 3.885066]
epoch:0 step:55 [D loss: 2.302995, acc.: 88.28%, op_acc: 35.16%] [G loss: 3.947450]
epoch:0 step:56 [D loss: 2.388173, acc.: 82.03%, op_acc: 31.25%] [G loss: 4.054560]
epoch:0 step:57 [D loss: 2.453156, acc.: 78.12%, op_acc: 32.03%] [G loss: 3.788085]
epoch:0 step:58 [D loss: 2.562096, acc.: 74.22%, op_acc: 33.59%] [G loss: 3.761415]
epoch:0 step:59 [D loss: 2.419969, acc.: 81.25%, op_acc: 36.72%] [G loss: 3.689951]
epoch:0 step:60 [D loss: 2.466167, acc.: 71.09%, op_acc: 39.06%] [G loss: 3.

epoch:0 step:149 [D loss: 1.628113, acc.: 96.09%, op_acc: 48.44%] [G loss: 4.642938]
epoch:0 step:150 [D loss: 1.528353, acc.: 96.88%, op_acc: 51.56%] [G loss: 4.211685]
epoch:0 step:151 [D loss: 1.577088, acc.: 97.66%, op_acc: 46.88%] [G loss: 4.714817]
epoch:0 step:152 [D loss: 1.504619, acc.: 99.22%, op_acc: 56.25%] [G loss: 4.509331]
epoch:0 step:153 [D loss: 1.660689, acc.: 100.00%, op_acc: 45.31%] [G loss: 4.475771]
epoch:0 step:154 [D loss: 1.316656, acc.: 97.66%, op_acc: 65.62%] [G loss: 4.391181]
epoch:0 step:155 [D loss: 1.292525, acc.: 99.22%, op_acc: 59.38%] [G loss: 4.591247]
epoch:0 step:156 [D loss: 1.418338, acc.: 100.00%, op_acc: 57.81%] [G loss: 4.635102]
epoch:0 step:157 [D loss: 1.340752, acc.: 99.22%, op_acc: 54.69%] [G loss: 4.058825]
epoch:0 step:158 [D loss: 1.522357, acc.: 97.66%, op_acc: 50.78%] [G loss: 4.769612]
epoch:0 step:159 [D loss: 1.297609, acc.: 100.00%, op_acc: 59.38%] [G loss: 4.421354]
epoch:0 step:160 [D loss: 1.730540, acc.: 96.09%, op_acc: 47.6

epoch:0 step:240 [D loss: 1.960624, acc.: 42.97%, op_acc: 60.16%] [G loss: 2.894954]
epoch:0 step:241 [D loss: 2.081457, acc.: 38.28%, op_acc: 59.38%] [G loss: 2.636191]
epoch:0 step:242 [D loss: 2.148110, acc.: 37.50%, op_acc: 54.69%] [G loss: 2.650867]
epoch:0 step:243 [D loss: 2.309648, acc.: 32.81%, op_acc: 57.03%] [G loss: 2.761945]
epoch:0 step:244 [D loss: 2.223727, acc.: 36.72%, op_acc: 53.91%] [G loss: 2.963344]
epoch:0 step:245 [D loss: 1.958644, acc.: 51.56%, op_acc: 60.16%] [G loss: 3.106137]
epoch:0 step:246 [D loss: 2.231929, acc.: 49.22%, op_acc: 50.00%] [G loss: 2.931849]
epoch:0 step:247 [D loss: 2.176180, acc.: 56.25%, op_acc: 49.22%] [G loss: 3.035604]
epoch:0 step:248 [D loss: 2.087957, acc.: 52.34%, op_acc: 52.34%] [G loss: 2.836115]
epoch:0 step:249 [D loss: 2.156623, acc.: 38.28%, op_acc: 59.38%] [G loss: 2.651988]
epoch:0 step:250 [D loss: 2.035970, acc.: 39.06%, op_acc: 57.81%] [G loss: 2.462709]
epoch:0 step:251 [D loss: 2.085191, acc.: 55.47%, op_acc: 54.69%]

epoch:0 step:340 [D loss: 1.502093, acc.: 46.09%, op_acc: 71.88%] [G loss: 1.652375]
epoch:0 step:341 [D loss: 1.163722, acc.: 53.91%, op_acc: 83.59%] [G loss: 1.439629]
epoch:0 step:342 [D loss: 1.307832, acc.: 58.59%, op_acc: 75.78%] [G loss: 1.602234]
epoch:0 step:343 [D loss: 1.150522, acc.: 60.16%, op_acc: 82.81%] [G loss: 1.548926]
epoch:0 step:344 [D loss: 1.159932, acc.: 65.62%, op_acc: 82.03%] [G loss: 1.608451]
epoch:0 step:345 [D loss: 1.186837, acc.: 54.69%, op_acc: 85.16%] [G loss: 1.435988]
epoch:0 step:346 [D loss: 1.167697, acc.: 65.62%, op_acc: 84.38%] [G loss: 1.445494]
epoch:0 step:347 [D loss: 1.290243, acc.: 57.81%, op_acc: 81.25%] [G loss: 1.572140]
epoch:0 step:348 [D loss: 1.370530, acc.: 54.69%, op_acc: 78.91%] [G loss: 1.512272]
epoch:0 step:349 [D loss: 1.267192, acc.: 53.91%, op_acc: 78.91%] [G loss: 1.484582]
epoch:0 step:350 [D loss: 1.107395, acc.: 57.81%, op_acc: 90.62%] [G loss: 1.482356]
epoch:0 step:351 [D loss: 1.162729, acc.: 58.59%, op_acc: 85.16%]

...

epoch:29 step:27655 [D loss: 0.751634, acc.: 46.09%, op_acc: 99.22%] [G loss: 0.682033]
epoch:29 step:27656 [D loss: 0.776317, acc.: 42.97%, op_acc: 98.44%] [G loss: 0.719007]
epoch:29 step:27657 [D loss: 0.758703, acc.: 37.50%, op_acc: 99.22%] [G loss: 0.708507]
epoch:29 step:27658 [D loss: 0.764499, acc.: 45.31%, op_acc: 99.22%] [G loss: 0.716726]
epoch:29 step:27659 [D loss: 0.728542, acc.: 45.31%, op_acc: 99.22%] [G loss: 0.707642]
epoch:29 step:27660 [D loss: 0.750957, acc.: 39.84%, op_acc: 99.22%] [G loss: 0.699524]
epoch:29 step:27661 [D loss: 0.798320, acc.: 46.88%, op_acc: 96.88%] [G loss: 0.737979]
epoch:29 step:27662 [D loss: 0.748414, acc.: 56.25%, op_acc: 97.66%] [G loss: 0.704772]
epoch:29 step:27663 [D loss: 0.751479, acc.: 52.34%, op_acc: 96.88%] [G loss: 0.687203]
epoch:29 step:27664 [D loss: 0.705988, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.693162]
epoch:29 step:27665 [D loss: 0.733316, acc.: 50.00%, op_acc: 99.22%] [G loss: 0.707012]
epoch:29 step:27666 [D loss: 0.

epoch:29 step:27748 [D loss: 0.746744, acc.: 48.44%, op_acc: 98.44%] [G loss: 0.711861]
epoch:29 step:27749 [D loss: 0.755818, acc.: 46.88%, op_acc: 99.22%] [G loss: 0.697110]
epoch:29 step:27750 [D loss: 0.711077, acc.: 50.78%, op_acc: 99.22%] [G loss: 0.732229]
epoch:29 step:27751 [D loss: 0.715843, acc.: 48.44%, op_acc: 99.22%] [G loss: 0.712791]
epoch:29 step:27752 [D loss: 0.731449, acc.: 55.47%, op_acc: 99.22%] [G loss: 0.733211]
epoch:29 step:27753 [D loss: 0.768043, acc.: 51.56%, op_acc: 96.88%] [G loss: 0.712216]
epoch:29 step:27754 [D loss: 0.721313, acc.: 56.25%, op_acc: 99.22%] [G loss: 0.722342]
epoch:29 step:27755 [D loss: 0.689618, acc.: 63.28%, op_acc: 99.22%] [G loss: 0.742638]
epoch:29 step:27756 [D loss: 0.696326, acc.: 59.38%, op_acc: 100.00%] [G loss: 0.716988]
epoch:29 step:27757 [D loss: 0.738220, acc.: 48.44%, op_acc: 98.44%] [G loss: 0.735098]
epoch:29 step:27758 [D loss: 0.787200, acc.: 55.47%, op_acc: 97.66%] [G loss: 0.735270]
epoch:29 step:27759 [D loss: 0.

epoch:29 step:27841 [D loss: 0.704586, acc.: 53.91%, op_acc: 99.22%] [G loss: 0.741642]
epoch:29 step:27842 [D loss: 0.705615, acc.: 53.91%, op_acc: 99.22%] [G loss: 0.737619]
epoch:29 step:27843 [D loss: 0.723210, acc.: 46.88%, op_acc: 98.44%] [G loss: 0.742891]
epoch:29 step:27844 [D loss: 0.742095, acc.: 50.78%, op_acc: 98.44%] [G loss: 0.735065]
epoch:29 step:27845 [D loss: 0.720568, acc.: 47.66%, op_acc: 99.22%] [G loss: 0.744067]
epoch:29 step:27846 [D loss: 0.718397, acc.: 50.00%, op_acc: 99.22%] [G loss: 0.742513]
epoch:29 step:27847 [D loss: 0.784901, acc.: 43.75%, op_acc: 98.44%] [G loss: 0.713574]
epoch:29 step:27848 [D loss: 0.715248, acc.: 40.62%, op_acc: 100.00%] [G loss: 0.738519]
epoch:29 step:27849 [D loss: 0.728268, acc.: 41.41%, op_acc: 99.22%] [G loss: 0.697073]
epoch:29 step:27850 [D loss: 0.710065, acc.: 54.69%, op_acc: 99.22%] [G loss: 0.714248]
epoch:29 step:27851 [D loss: 0.697784, acc.: 57.03%, op_acc: 99.22%] [G loss: 0.732961]
epoch:29 step:27852 [D loss: 0.

epoch:29 step:27937 [D loss: 0.703263, acc.: 49.22%, op_acc: 100.00%] [G loss: 0.706596]
epoch:29 step:27938 [D loss: 0.743324, acc.: 46.09%, op_acc: 97.66%] [G loss: 0.700733]
epoch:29 step:27939 [D loss: 0.727600, acc.: 53.12%, op_acc: 96.88%] [G loss: 0.716309]
epoch:29 step:27940 [D loss: 0.715840, acc.: 50.00%, op_acc: 99.22%] [G loss: 0.720719]
epoch:29 step:27941 [D loss: 0.704078, acc.: 58.59%, op_acc: 99.22%] [G loss: 0.727061]
epoch:29 step:27942 [D loss: 0.774054, acc.: 57.81%, op_acc: 98.44%] [G loss: 0.721340]
epoch:29 step:27943 [D loss: 0.716942, acc.: 43.75%, op_acc: 100.00%] [G loss: 0.697212]
epoch:29 step:27944 [D loss: 0.699732, acc.: 50.00%, op_acc: 100.00%] [G loss: 0.713883]
epoch:29 step:27945 [D loss: 0.689126, acc.: 58.59%, op_acc: 99.22%] [G loss: 0.726831]
epoch:29 step:27946 [D loss: 0.711736, acc.: 55.47%, op_acc: 99.22%] [G loss: 0.737422]
epoch:29 step:27947 [D loss: 0.743670, acc.: 63.28%, op_acc: 98.44%] [G loss: 0.705967]
epoch:29 step:27948 [D loss: 

epoch:29 step:28031 [D loss: 0.822760, acc.: 35.94%, op_acc: 97.66%] [G loss: 0.688157]
epoch:29 step:28032 [D loss: 0.763250, acc.: 32.81%, op_acc: 99.22%] [G loss: 0.686435]
epoch:29 step:28033 [D loss: 0.733990, acc.: 39.84%, op_acc: 99.22%] [G loss: 0.703066]
epoch:29 step:28034 [D loss: 0.678043, acc.: 63.28%, op_acc: 99.22%] [G loss: 0.724879]
epoch:29 step:28035 [D loss: 0.728962, acc.: 46.09%, op_acc: 100.00%] [G loss: 0.706499]
epoch:29 step:28036 [D loss: 0.746165, acc.: 41.41%, op_acc: 97.66%] [G loss: 0.710358]
epoch:29 step:28037 [D loss: 0.704580, acc.: 51.56%, op_acc: 100.00%] [G loss: 0.726179]
epoch:29 step:28038 [D loss: 0.740124, acc.: 46.09%, op_acc: 98.44%] [G loss: 0.721051]
epoch:29 step:28039 [D loss: 0.717356, acc.: 46.88%, op_acc: 99.22%] [G loss: 0.732035]
epoch:29 step:28040 [D loss: 0.708262, acc.: 56.25%, op_acc: 99.22%] [G loss: 0.727559]
epoch:29 step:28041 [D loss: 0.757274, acc.: 45.31%, op_acc: 97.66%] [G loss: 0.740282]
epoch:29 step:28042 [D loss: 0